# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

import os

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

# Y_train_8 = pd.read_csv('Data/Y_train_8.csv', header=None, delimiter=',').values
# Y_test_8 = pd.read_csv('Data/Y_test_8.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('Data/Y_train_std_21.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('Data/Y_test_std_21.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values

In [4]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

# train_y_8 = torch.tensor(Y_train_8, dtype=torch.float32)
# test_y_8 = torch.tensor(Y_test_8, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)

# Test

In [5]:
def train_and_predict_MGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)

    MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], covar_type = 'RQ', 
                                                                                     lr=0.05, num_iterations=10000, patience=10, device=Device)

    preds = Prediction.preds_for_one_model(MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    return preds



def evaluate_K(K_num, train_x, train_y, test_x, test_y, Device='cpu', PCA_trans='None'):
    results = Parallel(n_jobs=-1)(
        delayed(train_and_predict_MGP)(row_idx, train_x, train_y, test_x, test_y, K_num=K_num, Device=Device, PCA_trans=PCA_trans)
        for row_idx in range(test_y.shape[0])
    )
    full_test_preds_MGP = np.vstack(results)
    mse = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)
    return mse

K_values = [100, 200, 300, 400, 500]
results_dict = {}

for K in K_values:
    mse = evaluate_K(K, train_x, train_y, test_x, test_y)
    results_dict[K] = mse
    print(f"K_num = {K}, MSE = {mse}")

best_K = min(results_dict, key=results_dict.get)
print(f"K_num{best_K}, MSE{results_dict[best_K]}")

K_num = 100, MSE = 0.01571124792098999
K_num = 200, MSE = 0.010457956232130527
K_num = 300, MSE = 0.008705837652087212
K_num = 400, MSE = 0.01291919220238924
K_num = 500, MSE = 0.010832877829670906
K_num300, MSE0.008705837652087212


In [6]:
K_values = [100, 200, 300, 400, 500]
results_dict = {}

for K in K_values:
    mse = evaluate_K(K, train_x, train_y_21, test_x, test_y_21)
    results_dict[K] = mse
    print(f"K_num = {K}, MSE = {mse}")

best_K = min(results_dict, key=results_dict.get)
print(f"K_num{best_K}, MSE{results_dict[best_K]}")

K_num = 100, MSE = 0.02032635360956192
K_num = 200, MSE = 0.01132742129266262
K_num = 300, MSE = 0.011220529675483704
K_num = 400, MSE = 0.009855375625193119
K_num = 500, MSE = 0.008591480553150177
K_num500, MSE0.008591480553150177


In [7]:
def train_and_predict_MGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)

    MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], covar_type = 'RBF', 
                                                                                     lr=0.05, num_iterations=10000, patience=10, device=Device)

    preds = Prediction.preds_for_one_model(MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    return preds



def evaluate_K(K_num, train_x, train_y, test_x, test_y, Device='cpu', PCA_trans='None'):
    results = Parallel(n_jobs=-1)(
        delayed(train_and_predict_MGP)(row_idx, train_x, train_y, test_x, test_y, K_num=K_num, Device=Device, PCA_trans=PCA_trans)
        for row_idx in range(test_y.shape[0])
    )
    full_test_preds_MGP = np.vstack(results)
    mse = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)
    return mse

K_values = [100, 200, 300, 400, 500]
results_dict = {}

for K in K_values:
    mse = evaluate_K(K, train_x, train_y, test_x, test_y)
    results_dict[K] = mse
    print(f"K_num = {K}, MSE = {mse}")

best_K = min(results_dict, key=results_dict.get)
print(f"K_num{best_K}, MSE{results_dict[best_K]}")

K_num = 100, MSE = 0.012393452227115631
K_num = 200, MSE = 0.012121852487325668
K_num = 300, MSE = 0.01219131238758564
K_num = 400, MSE = 0.016201220452785492
K_num = 500, MSE = 0.016079779714345932
K_num200, MSE0.012121852487325668


In [8]:
K_values = [100, 200, 300, 400, 500]
results_dict = {}

for K in K_values:
    mse = evaluate_K(K, train_x, train_y_21, test_x, test_y_21)
    results_dict[K] = mse
    print(f"K_num = {K}, MSE = {mse}")

best_K = min(results_dict, key=results_dict.get)
print(f"K_num{best_K}, MSE{results_dict[best_K]}")

K_num = 100, MSE = 0.020978393033146858
K_num = 200, MSE = 0.012317252345383167
K_num = 300, MSE = 0.012270505540072918
K_num = 400, MSE = 0.010428703390061855
K_num = 500, MSE = 0.009725311771035194
K_num500, MSE0.009725311771035194


In [ ]:
K_values = [500, 600, 700]
results_dict = {}

for K in K_values:
    mse = evaluate_K(K, train_x, train_y_21, test_x, test_y_21)
    results_dict[K] = mse
    print(f"K_num = {K}, MSE = {mse}")
 
best_K = min(results_dict, key=results_dict.get)
print(f"K_num{best_K}, MSE{results_dict[best_K]}")

K_num = 500, MSE = 0.009708204306662083
K_num = 600, MSE = 0.008750076405704021
